In [54]:
# Import the required libraries and dependencies
import os
import pandas as pd
import plotly.express as px
import hvplot.pandas
from pathlib import Path
from dotenv import load_dotenv
from datetime import date

## Enable your Mapbox API access token

In [55]:
# Load the .env file into the notebook
load_dotenv()

# Read in your MAPBOX_API_KEY
mapbox_api_access_token = os.getenv('MAPBOX_API_ACCESS_TOKEN')

# Confirm the availability of your Mapbox API access token by checking its type
display(type(mapbox_api_access_token))

# Set your Mapbox API access token
px.set_mapbox_access_token(mapbox_api_access_token)

str

## Import the data 

In [56]:
# Import large kickstarter recent dataset
kickstarter_large_recent_df = pd.read_csv(
    Path('./Resources/kickstarter_large_dataset/ks-projects-201801.csv')
)

# Imported data has a space at the end of the column name
# Remove spaces in columns name
kickstarter_large_recent_df.columns = kickstarter_large_recent_df.columns.str.replace(' ','')

# Set the index as the ID
kickstarter_large_recent_df.set_index('ID', inplace=True)

# View head
kickstarter_large_recent_df.head()

,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usdpledged,usd_pledged_real,usd_goal_real
ID,,,,,,,,,,,,,,
1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95
1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00
1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00
1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00
1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.0,1283.0,19500.00


In [57]:
# Import small kickstarter live dataset
kickstarter_small_live_df = pd.read_csv(
    Path('./Resources/kickstarter_small_dataset/live.csv')
)

# Fix ID column
kickstarter_small_live_df.rename(columns={'Unnamed: 0': 'ID'}, inplace=True)

# Set the index as the ID
kickstarter_small_live_df.set_index('ID', inplace=True)

# View head
kickstarter_small_live_df.head()

,amt.pledged,blurb,by,country,currency,end.time,location,percentage.funded,state,title,type,url
ID,,,,,,,,,,,,
0,15823.0,"\n'Catalysts, Explorers & Secret Keepers: Wome...",Museum of Science Fiction,US,usd,2016-11-01T23:59:00-04:00,"Washington, DC",186,DC,"Catalysts, Explorers & Secret Keepers: Women o...",Town,/projects/1608905146/catalysts-explorers-and-s...
1,6859.0,\nA unique handmade picture book for kids & ar...,"Tyrone Wells & Broken Eagle, LLC",US,usd,2016-11-25T01:13:33-05:00,"Portland, OR",8,OR,The Whatamagump (a hand-crafted story picture ...,Town,/projects/thewhatamagump/the-whatamagump-a-han...
2,17906.0,\nA horror comedy about a repairman who was in...,Tessa Stone,US,usd,2016-11-23T23:00:00-05:00,"Los Angeles, CA",102,CA,Not Drunk Enough Volume 1!,Town,/projects/1890925998/not-drunk-enough-volume-1...
3,67081.0,\nThe Johnny Wander autobio omnibus you've all...,Johnny Wander,US,usd,2016-11-01T23:50:00-04:00,"Brooklyn, NY",191,NY,Our Cats Are More Famous Than Us: A Johnny Wan...,County,/projects/746734715/our-cats-are-more-famous-t...
4,32772.0,\nThe vision for this project is the establish...,Beau's All Natural Brewing Company,RW,cad,2016-11-18T23:05:48-05:00,"Kigali, Rwanda",34,Kigali Province,The Rwanda Craft Brewery Project,Town,/projects/beaus/the-rwanda-craft-brewery-proje...


In [58]:
# Import small kickstarter most backed dataset
kickstarter_small_most_backed_df = pd.read_csv(
    Path('./Resources/kickstarter_small_dataset/most_backed.csv')
)

# Fix ID column
kickstarter_small_most_backed_df.rename(columns={'Unnamed: 0': 'ID'}, inplace=True)

# Set the index as the ID
kickstarter_small_most_backed_df.set_index('ID', inplace=True)

# View head
kickstarter_small_most_backed_df.head()

,amt.pledged,blurb,by,category,currency,goal,location,num.backers,num.backers.tier,pledge.tier,title,url
ID,,,,,,,,,,,,
0,8782571.0,\nThis is a card game for people who are into ...,Elan Lee,Tabletop Games,usd,10000.0,"Los Angeles, CA",219382,"[15505, 202934, 200, 5]","[20.0, 35.0, 100.0, 500.0]",Exploding Kittens,/projects/elanlee/exploding-kittens
1,6465690.0,"\nAn unusually addicting, high-quality desk to...",Matthew and Mark McLachlan,Product Design,usd,15000.0,"Denver, CO",154926,"[788, 250, 43073, 21796, 41727, 21627, 12215, ...","[1.0, 14.0, 19.0, 19.0, 35.0, 35.0, 79.0, 79.0...",Fidget Cube: A Vinyl Desk Toy,/projects/antsylabs/fidget-cube-a-vinyl-desk-toy
2,5408916.0,\nBring Reading Rainbow’s library of interacti...,LeVar Burton & Reading Rainbow,Web,usd,1000000.0,"Los Angeles, CA",105857,"[19639, 14343, 9136, 2259, 5666, 24512, 4957, ...","[5.0, 10.0, 25.0, 30.0, 35.0, 50.0, 75.0, 100....","Bring Reading Rainbow Back for Every Child, Ev...",/projects/readingrainbow/bring-reading-rainbow...
3,5702153.0,\nUPDATED: This is it. We're making a Veronica...,Rob Thomas,Narrative Film,usd,2000000.0,"San Diego, CA",91585,"[5938, 8423, 11509, 22997, 23227, 1865, 7260, ...","[1.0, 10.0, 25.0, 35.0, 50.0, 75.0, 100.0, 150...",The Veronica Mars Movie Project,/projects/559914737/the-veronica-mars-movie-pr...
4,3336371.0,"\nAn adventure game from Tim Schafer, Double F...",Double Fine and 2 Player Productions,Video Games,usd,400000.0,"San Francisco, CA",87142,"[47946, 24636, 1090, 11530, 900, 148, 100, 10, 4]","[15.0, 30.0, 60.0, 100.0, 250.0, 500.0, 1000.0...",Double Fine Adventure,/projects/doublefine/double-fine-adventure


In [59]:
# Import indiegogo dataset
indiegogo_df = pd.read_csv(
    Path('./Resources/indiegogo.csv')
)

# Fix ID column
indiegogo_df.rename(columns={'project_id' : 'ID'}, inplace=True)

# Set the index as the ID
indiegogo_df.set_index('ID', inplace=True)

# View head
indiegogo_df.head()

,currency,category,year_end,month_end,day_end,time_end,amount_raised,funded_percent,in_demand,year_launch,...,apr,may,jun,jul,aug,sep,oct,nov,dec,tperiod
ID,,,,,,,,,,,,,,,,,,,,,
3936,USD,Transportation,2010,5,12,23:59:00,840,16.80%,False,2010,...,1,0,0,0,0,0,0,0,0,1
5109,USD,Human Rights,2010,7,2,23:59:00,250,20.83%,False,2010,...,0,0,1,0,0,0,0,0,0,2
5307,USD,Human Rights,2010,7,10,23:59:00,200,16.67%,False,2010,...,0,0,1,0,0,0,0,0,0,3
8731,USD,Photography,2010,10,9,23:59:00,500,25.00%,False,2010,...,0,0,0,0,0,1,0,0,0,4
9165,USD,Human Rights,2011,1,12,23:59:00,360,0.65%,False,2010,...,0,0,0,0,0,1,0,0,0,5


## Create dfs from selected columns

In [90]:
# List of kickstart and indiegogo columns to drop
kickstarter_cols_drop = ['category', 'backers', 'goal', 'pledged']
indiegogo_cols_drop = ['year_end',
'month_end',
'day_end',
'amount_raised',
'in_demand',
'year_launch',
'month_launch',
'day_launch',
'tagline',
'url',
'australia',
'canada',
'switzerland',
'denmark',
'western_europe',
'great_britain',
'hong_kong',
'norway',
'sweden',
'singapore',
'united_states',
'education',
'productivity',
'energy_greentech',
'wellness',
'comics',
'fashion_wearables',
'video_games',
'photography',
'tv_shows',
'dance_theater',
'phones_accessories',
'audio',
'film',
'transportation',
'art',
'environment',
'writing_publishing',
'music',
'travel_outdoors',
'health_fitness',
'tabletop_games',
'home',
'local_business',
'food_beverage',
'culture',
'human_rights',
'podcasts_vlogs',
'camera_gear',
'jan',
'feb',
'mar',
'apr',
'may',
'jun',
'jul',
'aug',
'sep',
'oct',
'nov',
'dec',
'tperiod']

# Selected columns kickstarter df
kickstarter_selected_cols_large_recent_df = kickstarter_large_recent_df.drop(kickstarter_cols_drop, axis=1)

# Kickstarter code to fix dates
kickstarter_selected_cols_large_recent_df['launched'] = pd.to_datetime(kickstarter_selected_cols_large_recent_df['launched'])
kickstarter_selected_cols_large_recent_df['deadline'] = pd.to_datetime(kickstarter_selected_cols_large_recent_df['deadline'])

# View kickstarter selected head
display(kickstarter_selected_cols_large_recent_df.head())

# Selected columns indiegogo df
indiegogo_selected_cols_df = indiegogo_df.drop(indiegogo_cols_drop, axis=1)

# Indiegogo code to fix dates
indiegogo_selected_cols_df['launched'] = pd.to_datetime(indiegogo_selected_cols_df['date_launch'] + ' ' + indiegogo_selected_cols_df['time_launch'])
indiegogo_selected_cols_df['deadline'] = pd.to_datetime(indiegogo_selected_cols_df['date_end'] + ' ' + indiegogo_selected_cols_df['time_end'])

# Drop the extra date info
indiegogo_selected_cols_df = indiegogo_selected_cols_df.drop(['time_end', 'time_launch', 'date_launch', 'date_end'], axis=1)

# View indiegogo selected head
display(indiegogo_selected_cols_df.head())

,name,main_category,currency,deadline,launched,state,country,usdpledged,usd_pledged_real,usd_goal_real
ID,,,,,,,,,,
1000002330,The Songs of Adelaide & Abullah,Publishing,GBP,2015-10-09,2015-08-11 12:12:28,failed,GB,0.0,0.0,1533.95
1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Film & Video,USD,2017-11-01,2017-09-02 04:43:57,failed,US,100.0,2421.0,30000.00
1000004038,Where is Hank?,Film & Video,USD,2013-02-26,2013-01-12 00:20:50,failed,US,220.0,220.0,45000.00
1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,USD,2012-04-16,2012-03-17 03:24:11,failed,US,1.0,1.0,5000.00
1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,USD,2015-08-29,2015-07-04 08:35:03,canceled,US,1283.0,1283.0,19500.00


,currency,category,funded_percent,title,state,amount_raised_usd,goal_usd,launched,deadline
ID,,,,,,,,,
3936,USD,Transportation,16.80%,Join the Electric Revolution!!!,0,840.0,5000.0,2010-04-21 22:38:42,2010-05-12 23:59:00
5109,USD,Human Rights,20.83%,Relief Trip to Haiti,0,250.0,1200.0,2010-06-10 17:47:35,2010-07-02 23:59:00
5307,USD,Human Rights,16.67%,"Out To Reach Leogane, Haiti 2010",0,200.0,1200.0,2010-06-18 09:49:01,2010-07-10 23:59:00
8731,USD,Photography,25.00%,The Transpersonal Papers: 1861-2010,0,500.0,2000.0,2010-09-09 13:50:48,2010-10-09 23:59:00
9165,USD,Human Rights,0.65%,Homeless Veterans need a Hand UP not Hand Out!,0,360.0,55000.0,2010-09-14 18:38:51,2011-01-12 23:59:00


In [91]:
# Rename the indiegogo columns to match the kickstarter df
indiegogo_selected_cols_df.rename(columns={'category' : 'main_category', 'title' : 'name', 'amount_raised_usd' : 'usdpledged', 'goal_usd' : 'usd_goal_real'}, inplace=True)
indiegogo_selected_cols_df

,currency,main_category,funded_percent,name,state,usdpledged,usd_goal_real,launched,deadline
ID,,,,,,,,,
3936,USD,Transportation,16.80%,Join the Electric Revolution!!!,0,840.000000,5000.0000,2010-04-21 22:38:42,2010-05-12 23:59:00
5109,USD,Human Rights,20.83%,Relief Trip to Haiti,0,250.000000,1200.0000,2010-06-10 17:47:35,2010-07-02 23:59:00
5307,USD,Human Rights,16.67%,"Out To Reach Leogane, Haiti 2010",0,200.000000,1200.0000,2010-06-18 09:49:01,2010-07-10 23:59:00
8731,USD,Photography,25.00%,The Transpersonal Papers: 1861-2010,0,500.000000,2000.0000,2010-09-09 13:50:48,2010-10-09 23:59:00
9165,USD,Human Rights,0.65%,Homeless Veterans need a Hand UP not Hand Out!,0,360.000000,55000.0000,2010-09-14 18:38:51,2011-01-12 23:59:00
...,...,...,...,...,...,...,...,...,...
2584905,USD,Film,14.47%,Totally Gay Productions,0,1351.000000,9338.0000,2020-07-16 00:00:00,2020-08-30 23:59:59
2599602,CHF,Music,2.17%,Lady Crow debut EP,0,80.288616,3697.5021,2020-07-17 00:00:00,2020-08-16 23:59:59
2595782,EUR,Fashion & Wearables,2.43%,"JOSA, the ultimate Venice-Inspired bracelet",0,81.420055,3346.0297,2020-07-17 00:00:00,2020-09-15 23:59:59


In [92]:
# Base column names for reordering
base_order = ['name', 'main_category', 'currency', 'deadline', 'launched', 'state', 'usd_goal_real', 'usdpledged']
kickstarter_order = base_order + ['usd_pledged_real', 'country']
indiegogo_order = base_order + ['funded_percent']

# Reorder each df
kickstarter_selected_cols_large_recent_df = kickstarter_selected_cols_large_recent_df[kickstarter_order]
indiegogo_selected_cols_df = indiegogo_selected_cols_df[indiegogo_order]

# Display
display(kickstarter_selected_cols_large_recent_df.head())
display(indiegogo_selected_cols_df.head())

,name,main_category,currency,deadline,launched,state,usd_goal_real,usdpledged,usd_pledged_real,country
ID,,,,,,,,,,
1000002330,The Songs of Adelaide & Abullah,Publishing,GBP,2015-10-09,2015-08-11 12:12:28,failed,1533.95,0.0,0.0,GB
1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Film & Video,USD,2017-11-01,2017-09-02 04:43:57,failed,30000.00,100.0,2421.0,US
1000004038,Where is Hank?,Film & Video,USD,2013-02-26,2013-01-12 00:20:50,failed,45000.00,220.0,220.0,US
1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,USD,2012-04-16,2012-03-17 03:24:11,failed,5000.00,1.0,1.0,US
1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,USD,2015-08-29,2015-07-04 08:35:03,canceled,19500.00,1283.0,1283.0,US


,name,main_category,currency,deadline,launched,state,usd_goal_real,usdpledged,funded_percent
ID,,,,,,,,,
3936,Join the Electric Revolution!!!,Transportation,USD,2010-05-12 23:59:00,2010-04-21 22:38:42,0,5000.0,840.0,16.80%
5109,Relief Trip to Haiti,Human Rights,USD,2010-07-02 23:59:00,2010-06-10 17:47:35,0,1200.0,250.0,20.83%
5307,"Out To Reach Leogane, Haiti 2010",Human Rights,USD,2010-07-10 23:59:00,2010-06-18 09:49:01,0,1200.0,200.0,16.67%
8731,The Transpersonal Papers: 1861-2010,Photography,USD,2010-10-09 23:59:00,2010-09-09 13:50:48,0,2000.0,500.0,25.00%
9165,Homeless Veterans need a Hand UP not Hand Out!,Human Rights,USD,2011-01-12 23:59:00,2010-09-14 18:38:51,0,55000.0,360.0,0.65%
